In [4]:
import torch
from dataset import TorsionPairDataset
from torch.utils.data import DataLoader

def collate_pad(batch):
    max_len = max(b["x0"].shape[1] for b in batch)
    B = len(batch)
    x0 = torch.zeros(B, 6, max_len)
    xref = torch.zeros(B, 6, max_len)
    valid_mask = torch.zeros(B, 6, max_len, dtype=torch.bool)
    esm = torch.zeros(B, max_len, 1280)
    pad_mask = torch.zeros(B, max_len, dtype=torch.bool)

    for i, b in enumerate(batch):
        L = b["x0"].shape[1]
        x0[i,:,:L] = b["x0"]
        xref[i,:,:L] = b["xref"]
        valid_mask[i,:,:L] = b["mask"]
        esm[i,:L,:] = b["esm"]
        pad_mask[i,:L] = True
        
    return {
        "x0": x0,
        "xref": xref,
        "valid_mask": valid_mask,
        "esm": esm,
        "pad_mask": pad_mask
    }

train_dataset = TorsionPairDataset()
train_loader = DataLoader(train_dataset, batch_size=32, collate_fn=collate_pad)

batch = next(iter(train_loader))
print(batch.keys())

dict_keys(['x0', 'xref', 'valid_mask', 'esm', 'pad_mask'])
